### Bokeh_Plots_Site.ipynb

Brian Dew, @bd_econ, March 17, 2018

----

Description: Retrieve GDP data and make a bar plot showing GDP growth.

In [1]:
#Import preliminaries
import requests
import pandas as pd
from datetime import datetime as dt
import time

# Local file with API key
import config
api_key = config.bea_key

# Bokeh settings
from bokeh.plotting import figure, show
import bokeh.models
import bokeh.plotting
from bokeh.models import ColumnDataSource, Span, Label, HoverTool, Range1d, BoxAnnotation
from bokeh.embed import components
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# US GDP

In [2]:
# Individual parameters of the request
uri = f'https://www.bea.gov/api/data/?&UserID={api_key}'
mdt = '&method=GetData&DataSetName=NIPA&TableName=T10101'
yr = ','.join(map(str, range(2010, 2019)))

# Construct URL from parameters above
url = f'{uri}{mdt}&Frequency=Q&Year={yr}&ResultFormat=json'

# Request parameter information from BEA API
r = requests.get(url).json()['BEAAPI']['Results']

# GDP data as a pandas dataframe
df = pd.DataFrame(
    [(pd.to_datetime(i['TimePeriod']), float(i['DataValue'])) 
     for i in r['Data'] if i['SeriesCode'] == 'A191RL'], 
    columns=['Date', 'Value']).set_index('Date')

# String dates
df['strdate'] = [f'{i.year} Q{i.quarter}' for i in df.index]

In [3]:
# GDP plot in Bokeh
source = ColumnDataSource({'x': df.index, 
                           'y': df['Value'], 
                           'Date': df['strdate']})

tooltips = """
    <div>
        <span style="font-size: 12px;">@Date:</span>
        <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 800.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

p = figure(plot_width=700, plot_height=300, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-2.6, 6.2))
p.vbar('x', width=width, bottom=0, top='y', 
       source=source, color='Red', alpha=1.0, 
       level='overlay')
p.add_layout(zero_line)
p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.toolbar.active_drag = None
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False))

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/gdp.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

# VIX

In [11]:
# VIX retrieved elsewhere, use data since 2015
df = pd.read_csv('C:/Working/USA/datafiles/safe/vix.csv', 
                 parse_dates=['date']).set_index('date')['vixclose']

df = df.loc['2015-01-01':]

In [12]:
# VIX plot in bokeh
source = ColumnDataSource({'x': df.index, 'y': df.values, 'Date': df.index.strftime('%b %d, %Y')})
tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=700,height=300,x_axis_type='datetime',
           tools=['pan, wheel_zoom, box_zoom, reset'], logo=None,
           toolbar_location=None)
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.line('x', 'y', source = source, line_width=2.5, color='Green', alpha=1.0, level='overlay')
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.add_tools(HoverTool(tooltips=tooltips, line_policy='nearest', show_arrow=False))
p.toolbar.active_drag = None
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/vix.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

# Germany and UK CPI

In [9]:
# CPI Inflation Data from the IMF
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = f"CompactData/IFS/M.DE+GB+IT+FR.PCPI_PC_CP_A_PT"
data = requests.get(f'{url}{key}').json()

df = pd.DataFrame({s['@REF_AREA'] : {pd.to_datetime(i['@TIME_PERIOD']) : 
     round(float(i['@OBS_VALUE']), 1) 
     for i in s['Obs'] if int(i['@TIME_PERIOD'][:4]) > 1991} 
     for s in data['CompactData']['DataSet']['Series']})

In [10]:
#Plotting CPI for Germany and the UK
source = ColumnDataSource({'x': df.index, 'yde': df.DE, 'yuk': df.GB, 'Date': df.index.strftime('%b %Y')})
tooltipsde = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">Germany</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@yde{1.1}%</span>
            </div>
"""
tooltipsuk = """
            <div>
                <span style="font-size: 12px; font-weight: bold;">United Kingdom</span><br>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@yuk{1.1}%</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=700,height=300,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None,
           toolbar_location=None, toolbar_sticky=False)
p.add_layout(hline)
p.sizing_mode = 'scale_width'
r1 = p.line('x', 'yde', source=source, legend='Germany', 
            line_width=2.5, color='#163172', alpha=1.0, level='overlay')
r2 = p.line('x', 'yuk', source=source, legend='United Kingdom', 
            line_width=2.5, color='#09a8fa', alpha=1.0, level='overlay')
p.legend.orientation = "horizontal"
p.legend.location = "top_center"
p.legend.border_line_alpha = 0
p.legend.background_fill_alpha = 0
p.legend.margin = 0
p.legend.padding = 0
p.legend.spacing = 15
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.add_tools(HoverTool(tooltips=tooltipsde, renderers=[r1], show_arrow=False))
p.add_tools(HoverTool(tooltips=tooltipsuk, renderers=[r2], show_arrow=False))
p.toolbar.active_drag = None
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/eu_cpi.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

# German CPI

In [8]:
# Germany CPI in Bokeh plot
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.DE.PCPI_PC_CP_A_PT' # adjust as needed
data = requests.get('{}{}'.format(url,key)).json()
# Navigate to observations in JSON data
obs = data['CompactData']['DataSet']['Series']['Obs']

# Create pandas dataframe from the observations
df = pd.DataFrame(obs)[['@OBS_VALUE','@TIME_PERIOD']]
df.columns = ['value', 'date']
df.loc[:,'value'] = df['value'].astype(float).round(1)
df = df.set_index(pd.to_datetime(df['date']))['value']

source = ColumnDataSource({'x': df.index, 'y': df.values, 'Date': df.index.strftime('%b %Y')})
tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=700,height=300,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None,
           toolbar_location=None, toolbar_sticky=False)
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.line('x', 'y', source = source, line_width=3, color='Orange', alpha=1.0, level='overlay')
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.add_tools(HoverTool(tooltips=tooltips, line_policy='nearest', show_arrow=False))
p.toolbar.active_drag = None
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/de_cpi.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

# MXN per USD

In [9]:
# USD MXN exchange rate
url = f'https://www.quandl.com/api/v3/datasets/CURRFX/USDMXN.csv?api_key={config.key}&start_date=2016-01-01'
df = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

source = ColumnDataSource({'x': df.index, 'y': df['Rate'], 'Date': df.index.format()})
tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y</span>
            </div>
"""
p = figure(width=700,height=300,x_axis_type='datetime',tools=['pan, wheel_zoom,box_zoom,reset'],
           logo=None, toolbar_location=None, toolbar_sticky=False)
#p.title.text = 'Mexican Pesos (MXN) per U.S. Dollar (USD)'
p.title.text_font_size='14pt'
#p.yaxis.axis_label = 'MXN/USD'
p.sizing_mode = 'scale_width'
p.line('x', 'y', source = source, line_width=3, color='Blue')
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False, point_policy='snap_to_data'))
p.toolbar.active_drag = None
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/usdmxn.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)

# New Orders

In [2]:
# New orders for capital goods excluding defense or aircraft
key = config.census_key
base = 'https://api.census.gov/data/timeseries/eits/m3'
param = 'cell_value,time_slot_id'
t = '&time=from+2015'
cat = '&category_code=NXA'
dtc = '&data_type_code=MPCNO'
oth = '&for=us&seasonally_adj=yes'

url = f'{base}?get={param}&key={key}{cat}{dtc}{t}{oth}'

r = requests.get(url).json()

df = pd.DataFrame({'date': [pd.to_datetime(i[4]) for i in r[1:]], 
                   'value': [float(i[0]) for i in r[1:]]}).sort_values('date')

In [7]:
# Manually input values from advanced report
df = df.set_index('date')
#df.at[pd.to_datetime('2018-01-01')] = -0.4
#df.at[pd.to_datetime('2018-02-01')] = 1.8

In [8]:
# New Orders plot in Bokeh
source = ColumnDataSource({'x': df.index, 
                           'y': df['value'], 
                           'Date': df.index.strftime('%b %Y')})

tooltips = """
    <div>
        <span style="font-size: 12px;">@Date:</span>
        <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
    </div>
"""

# Bar width determined by how many bars
width = ((df.index[-1] - df.index[0]).total_seconds() 
         * 800.0 / len(df.index))

# horizontal line at zero
zero_line = Span(location=0, dimension='width', 
                 line_color='gray', line_width=1)

# horizontal line at zero
tcja_start = time.mktime(dt(2017, 11, 15, 0, 0, 0).timetuple())*1000
tcja_label = time.mktime(dt(2017, 12, 1, 0, 0, 0).timetuple())*1000
tcja = Span(location=tcja_start, dimension='height', 
                 line_color='gray', line_dash='dashed', line_width=3)

p = figure(plot_width=700, plot_height=300, 
           x_axis_type='datetime', 
           tools=['pan, wheel_zoom, box_zoom, reset'], 
           logo=None,
           toolbar_location=None,  
           y_range=Range1d(-5.0, 3.5))
p.vbar('x', width=width, bottom=0, top='y', 
       source=source, color='Orange', alpha=1.0, 
       level='overlay')
#p.add_layout(zero_line)
citation = Label(x=tcja_label, y=2.5, #x_units='screen', y_units='screen',
                 text='TCJA', text_font_size='12px',#render_mode='css',
                 border_line_color='white', border_line_alpha=0.0,
                 background_fill_color='white', background_fill_alpha=0.0)
p.add_layout(citation)
p.renderers.extend([tcja, zero_line])#p.add_layout(tcja)
p.sizing_mode = 'scale_width'
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.toolbar.active_drag = None
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False))

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/neworders.html', 'w') as text_file:
    text_file.write(f'{script} {div}')
    
show(p)

# Employed Share of Population

In [18]:
# Employment rate
epop = pd.read_csv('C:/Working/Python/ChartBook/LNS12300060.csv')#.set_index('DATE')
epop = epop.set_index(pd.to_datetime(epop['DATE']))['LNS12300060']

source = ColumnDataSource({'x': epop.index, 'y': epop.values, 'Date': epop.index.strftime('%b %Y')})
tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}%</span>
            </div>
"""
p = figure(width=700,height=300,x_axis_type='datetime',tools=['pan, wheel_zoom,box_zoom,reset'],
           logo=None, toolbar_location=None, toolbar_sticky=False, y_range=Range1d(74, 83))
p.sizing_mode = 'scale_width'
p.add_layout(BoxAnnotation(left=pd.to_datetime('2001-04-01'), right=pd.to_datetime('2001-12-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.add_layout(BoxAnnotation(left=pd.to_datetime('2008-01-01'), right=pd.to_datetime('2009-07-01'), fill_alpha=0.1, line_alpha=0, fill_color='gray'))
p.line('x', 'y', source = source, line_width=3, color='Blue')
p.xgrid.grid_line_color = None
#p.grid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False, point_policy='snap_to_data'))
p.toolbar.active_drag = None
script, div = components(p)

with open('C:/Working/bdecon.github.io/plots/epop.html', 'w') as text_file:
    text_file.write('{} {}'.format(script, div))
    
show(p)